<a href="https://colab.research.google.com/github/djili/data-processing/blob/main/DiabeteDetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Detection de diabetes
Construire et évaluer un modèle de machine learning avec Spark ML.

1.  Chargez un DataFrame à partir d'un fichier CSV;
2.  Divisez les données en ensembles d'entraînement et de test;
3.  Construisez un pipeline de machine learning incluant des étapes de transformation et de modélisation;
4.  Entraînez le modèle et évaluez ses performances;

In [3]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler, StringIndexer
from pyspark.sql.functions import col
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [4]:
spark = SparkSession.builder.appName("Diabetes Detection").getOrCreate()

In [5]:
# Charger les données à partir d'un fichier CSV
data = spark.read.csv("/content/diabetes.csv", header=True, inferSchema=True)
data.printSchema()
data.show(5)

root
 |-- PatientID: integer (nullable = true)
 |-- Pregnancies: integer (nullable = true)
 |-- PlasmaGlucose: integer (nullable = true)
 |-- DiastolicBloodPressure: integer (nullable = true)
 |-- TricepsThickness: integer (nullable = true)
 |-- SerumInsulin: integer (nullable = true)
 |-- BMI: double (nullable = true)
 |-- DiabetesPedigree: double (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Diabetic: integer (nullable = true)

+---------+-----------+-------------+----------------------+----------------+------------+-----------+----------------+---+--------+
|PatientID|Pregnancies|PlasmaGlucose|DiastolicBloodPressure|TricepsThickness|SerumInsulin|        BMI|DiabetesPedigree|Age|Diabetic|
+---------+-----------+-------------+----------------------+----------------+------------+-----------+----------------+---+--------+
|  1354778|          0|          171|                    80|              34|          23|43.50972593|     1.213191354| 21|       0|
|  1147438|          

In [6]:
# Assembler les colonnes de caractéristiques
feature_columns = [col for col in data.columns if col != 'Diabetic']
assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")
data_prepared = assembler.transform(data)

In [7]:
# Diviser les données en ensembles d'entraînement et de test
train_data, test_data = data_prepared.randomSplit([0.8, 0.2])

In [8]:
# Entraîner un modèle de régression logistique
lr = LogisticRegression(featuresCol="features", labelCol="Diabetic")
lr_model = lr.fit(train_data)

In [9]:
# Faire des prédictions sur les données de test
predictions = lr_model.transform(test_data)
predictions.select("features", "Diabetic", "prediction").show(5)

+--------------------+--------+----------+
|            features|Diabetic|prediction|
+--------------------+--------+----------+
|[1000183.0,1.0,16...|       0|       0.0|
|[1000326.0,3.0,11...|       1|       1.0|
|[1001511.0,4.0,97...|       1|       1.0|
|[1002418.0,2.0,99...|       1|       1.0|
|[1003373.0,0.0,94...|       0|       0.0|
+--------------------+--------+----------+
only showing top 5 rows



In [10]:
# Correct the label column name to "Diabetic"
evaluator = BinaryClassificationEvaluator(labelCol="Diabetic", rawPredictionCol="prediction", metricName="areaUnderROC")
areaUnderROC = evaluator.evaluate(predictions)
print(f"ROC: {areaUnderROC}")

ROC: 0.7425296960290656


In [11]:
spark.stop()